<a href="https://colab.research.google.com/github/MuhammedShiway/AICrowd-Mosquito/blob/main/%5B03%5D_YOLO_Det_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive




---

# <center> PATHS </center>


---



In [ ]:
data_path = '/content/gdrive/MyDrive/Colab Notebooks/AI CROWD Mosquito/Dataset/Original /train_images'
train_csv_path = '/content/gdrive/MyDrive/Colab Notebooks/AI CROWD Mosquito/Dataset/Original /train.csv'
yolo_train = '/content/gdrive/MyDrive/Colab Notebooks/AI CROWD Mosquito/Dataset/YOLO/Train'
yolo_test = '/content/gdrive/MyDrive/Colab Notebooks/AI CROWD Mosquito/Dataset/YOLO/Test'

In [ ]:
# Paths
source_folder = data_path
train_folder = yolo_train
test_folder = yolo_test
csv_file = train_csv_path



---

# <center> IMPORT MODULES AND LIBRARIES </center>


---



In [ ]:
import os
import random
import shutil
import csv
from sklearn.model_selection import train_test_split



---

# <center> FOLDER CONTENT DELETER </center>


---



In [ ]:
def delete_folder_contents(folder_path):
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        try:
            if os.path.isfile(file_path) or os.path.islink(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path):
                shutil.rmtree(file_path)
        except Exception as e:
            print('Failed to delete %s. Reason: %s' % (file_path, e))

# delete_folder_contents('/path/to/your/folder')




---

# <center> ANNOT BBOX </center>


---



In [ ]:
def annot_bbox(images_folder, csv_file, im_names):
    with open(csv_file, 'r') as file:
        reader = csv.DictReader(file)
        for row in reader:
            image_name = row['img_fName']
            if image_name in im_names:
                image_path = os.path.join(images_folder, image_name)

                txt_file_path = os.path.splitext(image_path)[0] + '.txt'

                # Extract bounding box coordinates
                img_w = int(row['img_w'])
                img_h = int(row['img_h'])
                bbx_xtl = float(row['bbx_xtl'])
                bbx_ytl = float(row['bbx_ytl'])
                bbx_xbr = float(row['bbx_xbr'])
                bbx_ybr = float(row['bbx_ybr'])

                # Calculate YOLOv8 format values
                class_label = 0  
                x_center = (bbx_xtl + bbx_xbr) / (2 * img_w)
                y_center = (bbx_ytl + bbx_ybr) / (2 * img_h)
                width = abs((bbx_xbr - bbx_xtl) / img_w)
                height = abs((bbx_ytl - bbx_ybr) / img_h)

                # Write to txt file
                with open(txt_file_path, 'w') as txt_file:
                    txt_file.write(f"{class_label} {x_center} {y_center} {width} {height}")
        print("Txt files generated successfully.")


In [ ]:

with open(csv_file, 'r') as file:
    reader = csv.DictReader(file)
    image_names = [row['img_fName'] for row in reader]

train_names, test_names = train_test_split(image_names, test_size=0.2, random_state=42)

In [ ]:
# Generate txt files for the train folder
annot_bbox(train_folder, csv_file, train_names)

Txt files generated successfully.


In [ ]:
# Generate txt files for the test folder
annot_bbox(test_folder, csv_file, test_names)

Txt files generated successfully.


In [ ]:
folder_path = train_folder

image_count = 0
txt_count = 0
files = os.listdir(folder_path)
for file in files:
    file_path = os.path.join(folder_path, file)
    if os.path.isfile(file_path):
        if file.lower().endswith(".jpg") or file.lower().endswith(".jpeg") or file.lower().endswith(".png"):
            image_count += 1
        elif file.lower().endswith(".txt"):
            txt_count += 1

print("Number of images:", image_count)
print("Number of txt files:", txt_count)



Number of images: 6420
Number of txt files: 6420


In [ ]:
folder_path = test_folder

# Delete all txt files
for file in files:
    file_path = os.path.join(folder_path, file)
    if os.path.isfile(file_path) and file.lower().endswith(".txt"):
        os.remove(file_path)

print("Txt files deleted successfully.")

Txt files deleted successfully.




---

# <center> Subset 1000 </center>


---



In [ ]:
destination_folder = '/content/gdrive/MyDrive/Colab Notebooks/AI CROWD Mosquito/Dataset/YOLO/Train_100'
num_images_to_copy = 1000

os.makedirs(destination_folder, exist_ok=True)
image_files = [file for file in os.listdir(train_folder) if file.lower().endswith((".jpg", ".jpeg", ".png"))]
random.shuffle(image_files)

for i in range(num_images_to_copy):
    image_file = image_files[i]
    image_path = os.path.join(train_folder, image_file)
    txt_file = os.path.splitext(image_file)[0] + ".txt"
    txt_path = os.path.join(train_folder, txt_file)

    # Copy the image file to the destination folder
    shutil.copy(image_path, destination_folder)

    # Copy the txt file to the destination folder
    shutil.copy(txt_path, destination_folder)

print("Images and corresponding txt files copied successfully.")

Images and corresponding txt files copied successfully.




---

# <center> YOLO Training </center>


---



In [ ]:
# SETUP

%pip install ultralytics
import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.124 🚀 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)
Setup complete ✅ (12 CPUs, 83.5 GB RAM, 24.2/78.2 GB disk)


In [ ]:
%cd /content/gdrive/MyDrive/Colab Notebooks/AI CROWD Mosquito/Dataset/YOLO/Training

/content/gdrive/MyDrive/Colab Notebooks/AI CROWD Mosquito/Dataset/YOLO/Training


In [ ]:
!ls

data.yaml  runs  yolov8l.pt  yolov8n.pt  yolov8s.pt  yolov8x.pt




---

# <center> YOLO V8 </center>


---



In [ ]:
#TRAIN THE MODEL

!yolo task=detect mode=train model=yolov8n.pt data=data.yaml epochs=200 imgsz=640 plots=True



---

# <center> YOLO V5 </center>


---



In [ ]:
#clone YOLOv5 and
!git clone https://github.com/ultralytics/yolov5  
%cd yolov5
%pip install -qr requirements.txt 
%pip install -q roboflow

import torch
import os
from IPython.display import Image, clear_output  

print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

Cloning into 'yolov5'...
remote: Enumerating objects: 16008, done.
remote: Total 16008 (delta 0), reused 0 (delta 0), pack-reused 16008
Receiving objects: 100% (16008/16008), 14.51 MiB | 15.41 MiB/s, done.
Resolving deltas: 100% (11012/11012), done.
/content/yolov5
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 612.6/612.6 kB 58.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.3/155.3 kB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.6/55.6 kB 8.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5

In [ ]:
!python train.py --img 384 --batch 16 --epochs 150 --data data.yaml --weights yolov5s.pt --cache

train: weights=yolov5s.pt, cfg=, data=data.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=150, batch_size=16, imgsz=384, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=runs/train, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v7.0-187-g0004c74 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)

hyperparameters: lr0=0.01, lrf=0.01, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv_h=0.015, hsv_s=0.7, 